# Inference

## Copy tokenizer

In [8]:
from transformers import AutoTokenizer

# 1. Specify the name of the Hugging Face model
model_name = "Qwen/Qwen3-0.6B" 

# 2. Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. Define the local directory where you want to save the tokenizer
local_directory = "/home/ubuntu/environment/ml/qwen/compiled_model"

# 4. Save the tokenizer to the local directory
tokenizer.save_pretrained(local_directory)

print(f"Tokenizer for '{model_name}' saved to '{local_directory}'")

Tokenizer for 'Qwen/Qwen3-0.6B' saved to '/home/ubuntu/environment/ml/qwen/compiled_model'


# Compilation

After completing the fine-tuning process, the next step is to compile the trained model for AWS Trainium inference using the Hugging Face Optimum Neuron toolchain.
Neuron compilation optimizes the model graph and converts it into a Neuron Executable File Format (NEFF), enabling efficient execution on NeuronCores.

In [3]:
!optimum-cli export neuron \
  --model "/home/ubuntu/environment/distillation/Qwen3-0.6B-finetuned" \
  --task text-generation \
  --sequence_length 2048 \
  --batch_size 1 \
  /home/ubuntu/environment/ml/qwen/compiled_model

/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise.py:68: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  component, error = import_nki(config)
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise

# Inference

We will install the Optimum Neuron vllm library.  Then, run inference using the compiled model.

In [4]:
%pip install -q optimum-neuron[vllm]

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached sentencepiece-0.2.1-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (10 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl.metadata (4.2 kB)
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 43.8 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 176.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 52.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 72.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 73.8 MB/s  0:00:01m0:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 9.7 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━

Now you can run the batch inference example below

In [10]:
import os
from vllm import LLM, SamplingParams
llm = LLM(
    model="/home/ubuntu/environment/ml/qwen/compiled_model", #local compiled model
    max_num_seqs=1,
    max_model_len=2048,
    tensor_parallel_size=2,)

def create_conversation(sample):
    return f"""<|im_start|>system
    You are a sentiment classifier. You take input strings and return the sentiment of POSITIVE, NEGATIVE, or NEUTRAL. Only return the sentiment.
    <|im_start|>user
    {sample}
    <|im_start|>assistant"""

prompts = []
with open('/home/ubuntu/environment/distillation/data/dataset.txt', 'r') as f:
    for line in f:
        if line.strip():
            prompts.append( create_conversation(line.strip()) )
print(prompts)
sampling_params = SamplingParams(max_tokens=2048, temperature=0.8)
outputs = llm.generate(prompts, sampling_params)

print("#########################################################")

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, \n\n Generated text: {generated_text!r} \n")

INFO 10-31 01:38:28 [utils.py:328] non-default args: {'max_model_len': 2048, 'tensor_parallel_size': 2, 'max_num_seqs': 1, 'disable_log_stats': True, 'model': '/home/ubuntu/environment/ml/qwen/compiled_model'}
INFO 10-31 01:38:28 [__init__.py:742] Resolved architecture: Qwen3ForCausalLM
INFO 10-31 01:38:28 [__init__.py:1815] Using max model len 2048
INFO 10-31 01:38:28 [llm_engine.py:221] Initializing a V0 LLM engine (v0.10.2) with config: model='/home/ubuntu/environment/ml/qwen/compiled_model', speculative_config=None, tokenizer='/home/ubuntu/environment/ml/qwen/compiled_model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cpu, decoding_config=DecodingConfig(backend=

INFO:Neuron:Loading sharded checkpoint from /home/ubuntu/environment/ml/qwen/compiled_model/checkpoint/weights


WARNING 10-31 01:38:35 [__init__.py:3804] Current vLLM config is not set.
WARNING 10-31 01:38:35 [__init__.py:3804] Current vLLM config is not set.
INFO 10-31 01:38:35 [executor_base.py:114] # neuron blocks: 2, # CPU blocks: 0
INFO 10-31 01:38:35 [executor_base.py:119] Maximum concurrency for 2048 tokens per request: 2.00x
INFO 10-31 01:38:35 [llm_engine.py:420] init engine (profile, create kv cache, warmup model) took 0.00 seconds
INFO 10-31 01:38:35 [llm.py:295] Supported_tasks: ['generate']
INFO 10-31 01:38:35 [__init__.py:36] No IOProcessor plugins requested by the model
['<|im_start|>system\n    You are a sentiment classifier. You take input strings and return the sentiment of POSITIVE, NEGATIVE, or NEUTRAL. Only return the sentiment.\n    <|im_start|>user\n    The service at this restaurant exceeded all my expectations!\n    <|im_start|>assistant', "<|im_start|>system\n    You are a sentiment classifier. You take input strings and return the sentiment of POSITIVE, NEGATIVE, or NE

Adding requests:   0%|          | 0/100 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

#########################################################
Prompt: '<|im_start|>system\n    You are a sentiment classifier. You take input strings and return the sentiment of POSITIVE, NEGATIVE, or NEUTRAL. Only return the sentiment.\n    <|im_start|>user\n    The service at this restaurant exceeded all my expectations!\n    <|im_start|>assistant', 

 Generated text: '\nassistant\nThe service at this restaurant exceeded all my expectations!' 

Prompt: "<|im_start|>system\n    You are a sentiment classifier. You take input strings and return the sentiment of POSITIVE, NEGATIVE, or NEUTRAL. Only return the sentiment.\n    <|im_start|>user\n    I can't believe how rude the staff was today.\n    <|im_start|>assistant", 

 Generated text: '\nassistant\nThe report contains data from the last fiscal year.' 

Prompt: '<|im_start|>system\n    You are a sentiment classifier. You take input strings and return the sentiment of POSITIVE, NEGATIVE, or NEUTRAL. Only return the sentiment.\n    <|im_sta

You can start an inference endpoint on the current instance, like

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
    --model="/home/ubuntu/environment/ml/qwen/compiled_model" \
    --max-num-seqs=1 \
    --max-model-len=2048 \
    --tensor-parallel-size=2 \
    --port=8080 \
    --device "neuron"

And query the inference endpoint like

In [ ]:
curl 127.0.0.1:8080/v1/completions \
    -H 'Content-Type: application/json' \
    -X POST \
    -d '{"prompt":"<|im_start|>system\n    You are a sentiment classifier. You take input strings and return the sentiment of POSITIVE, NEGATIVE, or NEUTRAL. Only return the sentiment.\n    <|im_start|>user\n    The service at this restaurant exceeded all my expectations!\n    <|im_start|>assistant", "temperature": 0.8, "max_tokens":128}'
